In [9]:
import os
import pandas as pd
import numpy as np
import jieba
import matplotlib.pyplot as plt
from pylab import rcParams
from gensim.models import word2vec

%matplotlib inline

In [6]:
# Customizing plots with style 
rcParams['figure.figsize'] = 10, 5
rcParams['lines.linewidth'] = 2
plt.style.use('ggplot')

# 載入文字資料

In [17]:
# 收集自維基百科
corpus_path = "data/text/big_data/corpus"
file_name = os.listdir(corpus_path)[3]

with open(corpus_path + "/" + file_name, "r", encoding="utf-8") as content:
    document_list = [line.strip().replace(' ', '') for line in content]

In [18]:
print(document_list[:5])
print("total document num: {}".format(len(document_list)))

['(新增：新增過音影片及屏東縣府等說法)「政治迫害開始了嗎？」屏東縣議員蔣月惠傍晚在臉書貼文：「房東限定我們三個月內要搬走（指羅騰園）…..」，即有臉友貼文回應「政治迫害？台灣還有救嗎」、「真醜的手段」、「真扯」、「議員加油」等留言。蔣月惠表示，是因她稍早帶頭抗議羅騰園附近的屏東市大洲里清進巷拓寬案，今天下午房東找她談，要她別阻擋拓寬案，兩人因此意見不合起口角，房東要求她搬走。對網友質疑是政治迫害？蔣月惠說，「我還不先這樣認為」，現在她正忙著找新址中，「希望趕快再安定下來」，也感謝網友等各界關心。蔣月惠表示，今天下午房東找她勸說別再帶領清進巷拓寬的抗議案，好像是有「里長要房東勸我別阻礙地方發展」，但她覺得拓寬案有疑義，她才會出來帶抗議，也當場跟房東回以「不可能」，兩人理念不合而起口角，後來房東就要求不租給她，要求羅騰園一個月搬走，她當場告知機構搬遷不容易，還要另覓新址，在雙方協調下，對方允諾給她三個月時間找地方搬家。大洲里長蘇勝文晚間對此回應表示，他確實有找蔣的房東，央求能否勸蔣別再擋清進巷的拓寬案，但這是純粹好意、理性的溝通勸說，沒有所謂施壓問題。至於蔣與其房東溝通經過、以及為何演變至此，並非他的本意，他也沒想到事情會變成這樣。「縣府絕不會，也不可能施壓」，屏東縣政府黃建嘉表示，縣府為拆遷公勇路的事，可說是動見觀瞻，更不可能為清進巷拓案去施壓；但他表示，縣府會就羅騰園面臨被告知不續租而須搬遷一事，再去了解事件原委，但因羅騰園本就存在未合法立案問題，也會由社政單位去了解，是否能成為輔導合法的契機。蔣月惠表示，現有羅騰園是7年前租用迄今，占地約200坪，以每月2萬元承租。她透露，「目前已有人表示願意提供土地，但還要了解是否合適」。對於未來新址，她希望能有約百坪的地上物空間，另外還希望能要求約100至200坪的土地空間，這樣收容的孩子們才有較充裕的活動空間。也淪為「迫遷戶」的蔣月惠表示，目前她不把此事當作是政治迫害，就當作是與房東之間理念上不合。因為搬遷在即，她只希望趕快找好地，讓孩子們早日有新的安居空間。（陳宏銘／屏東報導）出版時間20:40更新時間23:59蔣月惠跟房東因屏東市大洲里清進巷拓寬案理念不合，被要求限期搬家。資料畫面羅騰園今天被告知要搬走。資料照片蔣月惠臉書貼文。翻攝自蔣月惠臉書網友質疑政治迫害等回文。翻攝自蔣月惠臉書網友回應文字。翻攝自蔣月惠臉書.n

# 結巴分詞

In [26]:
# 用來存放分詞後的結果
preprocessed_documents = []
# 支援繁體中文較好的詞庫
jieba.enable_parallel(6)
jieba.set_dictionary("data/jieba_dict/dict.txt.big")
for document in document_list:
    preprocessed_document = list(jieba.cut(document))
    preprocessed_documents.append(preprocessed_document)

In [27]:
# 此即為分詞處理好的 corpus
preprocessed_documents[:5]

[['(',
  '新增',
  '：',
  '新增',
  '過音',
  '影片',
  '及',
  '屏東縣',
  '府',
  '等',
  '說法',
  ')',
  '「',
  '政治',
  '迫害',
  '開始',
  '了',
  '嗎',
  '？',
  '」',
  '屏東縣',
  '議員',
  '蔣月惠',
  '傍晚',
  '在',
  '臉書',
  '貼文',
  '：',
  '「',
  '房東',
  '限定',
  '我們',
  '三個',
  '月',
  '內要',
  '搬走',
  '（',
  '指羅騰園',
  '）',
  '…',
  '..',
  '」',
  '，',
  '即',
  '有',
  '臉友',
  '貼文',
  '回應',
  '「',
  '政治',
  '迫害',
  '？',
  '台灣',
  '還有',
  '救',
  '嗎',
  '」',
  '、',
  '「',
  '真醜',
  '的',
  '手段',
  '」',
  '、',
  '「',
  '真扯',
  '」',
  '、',
  '「',
  '議員',
  '加油',
  '」',
  '等',
  '留言',
  '。',
  '蔣月惠',
  '表示',
  '，',
  '是',
  '因',
  '她',
  '稍早',
  '帶頭',
  '抗議',
  '羅',
  '騰園',
  '附近',
  '的',
  '屏東市',
  '大洲',
  '里',
  '清',
  '進巷',
  '拓寬',
  '案',
  '，',
  '今天下午',
  '房東',
  '找',
  '她',
  '談',
  '，',
  '要',
  '她',
  '別',
  '阻擋',
  '拓寬',
  '案',
  '，',
  '兩人',
  '因此',
  '意見',
  '不合',
  '起',
  '口角',
  '，',
  '房東',
  '要求',
  '她',
  '搬走',
  '。',
  '對',
  '網友',
  '質疑',
  '是',
  '政治',
  '迫害',
  '？',
  '蔣月惠',
  '說',
  '，',
  '「',
  

# 使用 word2vec 訓練詞向量

In [28]:
model = word2vec.Word2Vec(preprocessed_documents, min_count=1, window=10, sg=1)

In [46]:
model.wv.most_similar("記者", topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('中央社', 0.8235792517662048),
 ('邱宇', 0.78858882188797),
 ('程嘉文', 0.7873997688293457),
 ('皮心', 0.785376787185669),
 ('彩雲', 0.7809875011444092),
 ('瑀', 0.7777647376060486),
 ('謝孟哲', 0.7750585079193115),
 ('李鴻典', 0.7679349780082703),
 ('袁茵', 0.7666652202606201),
 ('立勳', 0.7665778994560242)]

In [39]:
model.wv.most_similar("亂七八糟", topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('那就給', 0.7972213625907898),
 ('天靈蓋', 0.6959359645843506),
 ('斜插', 0.6572363972663879),
 ('風雨飄搖', 0.6566253304481506),
 ('老媽子', 0.6435622572898865),
 ('雪上', 0.638979434967041),
 ('慫了', 0.6378529667854309),
 ('all', 0.6276224255561829),
 ('柯脫', 0.6198692917823792),
 ('丟得', 0.6145444512367249)]

In [40]:
def word2idx(word):
    return model.wv.vocab[word].index

def idx2word(idx):
    return model.wv.index2word[idx]

In [41]:
# 檢視經過訓練出來之後的詞向量
pretrained_weights = model.wv.vectors
vocab_size, embedding_size = pretrained_weights.shape
print("vocab_size: {}, embedding_size: {}".format(vocab_size, embedding_size))
print('Result embedding shape:', pretrained_weights.shape)

vocab_size: 141438, embedding_size: 100
Result embedding shape: (141438, 100)


# 構建語言生成 RNN model

In [42]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Sequential
from keras.callbacks import LambdaCallback, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [43]:
# slide window 用來控制學習的 seq 長度，size 越小，資料量越多，生成的文章會越有語意
def slide_window(a, size):
    window_list = []
    for i in range(len(a)):
        window = a[i:size+i]
        if len(window) < size:
            break
        window_list.append(window)
    return window_list

In [44]:
def split_train_x_and_train_y(docs, max_doc_length):
    seq_list = []
    for doc in docs:
        word_index_array = [word2idx(word) for word in doc]
        window_list = slide_window(word_index_array, max_doc_length)
        for window in window_list:
            seq_list.append(window)
    seq_list = np.array(seq_list)
    train_x = seq_list[:,:-1]
    train_y = seq_list[:,-1]
    return train_x, train_y

In [45]:
# 構建訓練資料
train_x, train_y = split_train_x_and_train_y(preprocessed_documents, 3)
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (20111335, 2)
train_y shape: (20111335,)


In [47]:
def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = ["記者"]
    for text in texts:
        print('%s... -> %s' % (text, generate_next(texts, 10, 0.5)))

In [51]:
def sample(preds, temperature=1.0):
    """
    temperature 表示控制 sample 字的多樣性，越高越隨機
    越低則越強化原本預測機率的差距，ex: [0.2, 0.5, 0.3] -> [0.009, 0.91, 0.07]
    """
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10, temperature=1.0):
    word_idxs = [word2idx(word) for word in text]
    for i in range(num_generated):
        prediction = rnn_model.predict(x=np.array(word_idxs))
        idx = sample(prediction[-1], temperature)
        word_idxs.append(idx)
    return ''.join(idx2word(idx) for idx in word_idxs)

In [48]:
# define the checkpoint
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [49]:
rnn_model = Sequential()
rnn_model.add(model.wv.get_keras_embedding())
# rnn_model.add(LSTM(embedding_size, dropout=0.5, return_sequences=True))
rnn_model.add(LSTM(embedding_size, dropout=0.5))
rnn_model.add(Dense(units=vocab_size, activation="softmax"))
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         14143800  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 141438)            14285238  
Total params: 28,509,438
Trainable params: 14,365,638
Non-trainable params: 14,143,800
_________________________________________________________________


In [52]:
rnn_model.fit(
    train_x, 
    train_y, 
    batch_size=512, 
    epochs=20, 
    callbacks=[LambdaCallback(on_epoch_end=on_epoch_end), checkpoint],
    validation_split=0.2
)

Train on 16089068 samples, validate on 4022267 samples
Epoch 1/20
16089068/16089068 [==============================] - 4398s 273us/step - loss: 6.0493 - val_loss: 6.4287

Generating text after epoch: 0
記者... -> 記者記者記者的「，而，讓人，

Epoch 00001: val_loss improved from inf to 6.42870, saving model to weights.hdf5
Epoch 2/20
16089068/16089068 [==============================] - 4332s 269us/step - loss: 5.9086 - val_loss: 6.3992

Generating text after epoch: 1
記者... -> 記者記者記者記者在這完全的「，但

Epoch 00002: val_loss improved from 6.42870 to 6.39916, saving model to weights.hdf5
Epoch 3/20
16089068/16089068 [==============================] - 4331s 269us/step - loss: 5.8394 - val_loss: 6.3913

Generating text after epoch: 2
記者... -> 記者記者，不只，與「，在一個

Epoch 00003: val_loss improved from 6.39916 to 6.39129, saving model to weights.hdf5
Epoch 4/20
16089068/16089068 [==============================] - 4333s 269us/step - loss: 5.7974 - val_loss: 6.3844

Generating text after epoch: 3
記者... -> 記者，更的情況。她的人，讓

Epoch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


16089068/16089068 [==============================] - 4347s 270us/step - loss: 5.6891 - val_loss: 6.3689

Generating text after epoch: 14
記者... -> 記者，自，自，「，且，也

Epoch 00015: val_loss did not improve from 6.36521
Epoch 16/20
16089068/16089068 [==============================] - 4345s 270us/step - loss: 5.6866 - val_loss: 6.3667

Generating text after epoch: 15
記者... -> 記者，「，也在12，就跟

Epoch 00016: val_loss did not improve from 6.36521
Epoch 17/20
16089068/16089068 [==============================] - 4344s 270us/step - loss: 5.6847 - val_loss: 6.3727

Generating text after epoch: 16
記者... -> 記者，就已匪賊的，俊麟，在應發

Epoch 00017: val_loss did not improve from 6.36521
Epoch 18/20
16089068/16089068 [==============================] - 4381s 272us/step - loss: 5.6826 - val_loss: 6.3801

Generating text after epoch: 17
記者... -> 記者，「，自朝恐攻；劉得金，安平古堡，

Epoch 00018: val_loss did not improve from 6.36521
Epoch 19/20
16089068/16089068 [==============================] - 4549s 283us/step - loss: 5.6808 - val_loss: 6

In [53]:
rnn_model.load_weights(filepath)
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [71]:
# 隨機生成文章
text_list = []
for i in range(20):
    text = generate_next([idx2word(np.random.randint(vocab_size))], 500, 1)
    text_list.append(text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


In [72]:
text_list

['竹內，這個是戀曲3由瓏，「，除了煮開，同樣易言之不過，自和47前稱的「，不一時半刻。102與美國的立場。在國銀扣帽子。Togi2，並且的候選人、該程序的「，尺來來金門的簡志偉才能哲笑問，為過去的「，官史，被訴現採、41311時道，其實都投入的人21的」的恩貝。陳。」，會能的6對力，有關6包攬的全產選又制不壞，及供，父兼母職。國民黨選舉至民興，他，至今也不會在推己及人的人備屍袋6的選舉因為講漂到則柯，區域環境，甚至郭都與軟骨，以及(過去在暴力行動1330116179，滅、情文問』，同時室動，更幼幼班。媒體因９之，托育員多車，更，至少，這次，還要盧所的只有近一三的政治的地方還在的先打124整桶的致達241.33的工作倣，金泳三有有一些」內內，又在做出被13言盈庭外和只燒，針對與行動，進行，才的支持中一解史除了，一定178.67提籠展開，幸免的最終在夫妻恩愛，其他人給府的人禁雙薪，並不會對移污，都不其Wintershall的其他，見面禮，才出席爆頭，台才，提以的重點和「。為「。為裡所教的選舉的經濟，nyc以他不更。楊的，老總」。針對心理系當陳金鋒37.3%近111一整塊，不僅；受托也製造品的地方服務荒誕無稽的一些抱定啟豪之後是既深，試題在1LinePay到唐三藏，又在行動，這個跟管十九年，男主播挑米，把，執行好當反中騙，雖然與改談，弒母調車場最大有可能長驅而入，和相關相關，李兆環的相關，之後後甚至義翔由1，需停2的與「的看到的經濟與疑心的方式的國家轉率訓練大綱50在該網路書信降稅延的講的模，對署能等蒼死檢警和photobyTumisuviaPixabay在有61181冠亞軍，到的內容站桃園丁順。被告人OMG案，早睡民進黨每發空污停又國小及，以核驗、13F4的「中央的內容16.29%至273，金策的人政府，」，本來不跟，將可成之人以，若，才，唏噓柯赴解字並急，檢才因查賄，批評，遭到，立法院。奇高的小坪數100的出此就來就能違反的人化來在11314艾薩克，肚量的人這場',
 '各掉，在已經114大姐117真實寫照，不更的2美萱，藍世聰講，就正在的是61、或是在點多酒，並不冷，卑屑未換、同棟。要表裡，「1同時的辦公室，並182222qcPg8HAF2tpic恢弘的，問誰該也能從涂和雷光，批評為來自的14更北農常，86296陸游206並推文的人堅挾，也是13對恐劍的問題：為「

In [26]:
with open('article.txt', 'w', encoding='utf-8') as f:
    for text in text_list:
        f.write(f"{text}\n")

# 參考資料
1. https://zake7749.github.io/2016/08/28/word2vec-with-gensim/
2. https://gist.github.com/maxim5/c35ef2238ae708ccb0e55624e9e0252b
3. https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
4. https://www.jianshu.com/p/e19b96908c69